In [8]:
import overpass, osmapi, time

In [9]:
# open condos file
f = open("condos.csv","r").readlines()

# start not found condo file
j = open("condos-left.csv","w")

# set username
username = 'schleuss'

# crack open APIs
api = overpass.API()
api2 = osmapi.OsmApi(passwordfile = "/Users/jschleuss/Documents/osm.password")

In [44]:
# find id by ain
def findBuildings(ain):
    ain = ain.replace('\n','')
    print 'way["lacounty:ain"="'+ain+'"]'
    
    # example condo parcel
#     response = api.Get('((way["lacounty:ain"="'+str(ain)+'"]);(relation["lacounty:ain"="'+str(ain)+'"]););')
    response = api.Get('(relation["lacounty:ain"="7563006014"]);')
    
    
    print str(len(response['features'])) + ' total features found for AIN ' + ain
    print response
    

In [45]:
# testing to find buildings
selain = f[2]
findBuildings('7563006014')

way["lacounty:ain"="7563006014"]
0 total features found for AIN 7563006014
{"features": [], "type": "FeatureCollection"}


In [16]:
def updateBuildings(ain):

    # WILL NEED TO LOOP THROUGH POSSIBLE MULTIPLE BUILDINGS
    ain = ain.replace('\n','')
    
    # get features
    response = api.Get('(way["lacounty:ain"="'+str(ain)+'"];relation["lacounty:ain"="'+str(ain)+'"];)')

    # open a changeset
    api2.ChangesetCreate({u"comment": u"Removing building:units from condo and adding apartments type", u'source':'L.A. County Assessor', u'created_by':username})    
    
    # loop through all features
    for feature in response['features']:
        objid = feature['id']
        obj = api2.WayGet(objid) # get object


        # delete unit count and change building to residential 
        if 'building:units' in obj['tag']:
            del obj['tag']['building:units']
            print 'updated units for ' + str(objid)
        if obj['tag']['building'] != 'apartments':
            obj['tag']['building'] = 'apartments'
            print 'updated building type for ' + str(objid)

        obj = api2.WayUpdate(obj) # update the object
        
        print "updating " + str(objid)
        # wait a beat
        time.sleep(1)
        
    api2.ChangesetClose()
        
        
#     # alt if way gives nothing
#     if len(response['features']) == 0:
#         response = api.Get('way["lacounty:ain"="'+ain+'"]')
#         objid = response['features'][0]['id']

#     else:
#         objid = response['features'][0]['id']
        
#     obj = api2.WayGet(objid) # get object

#     open a changeset
#     api2.ChangesetCreate({u"comment": u"Removing building:units from condo and adding apartments type", u'source':'L.A. County Assessor', u'created_by':username})
    


In [19]:
# api2.ChangesetClose()
# testing to update buildings
updateBuildings(selain)

''
updated 407337799
updated units for 407338314
updated building type for 407338314
updated 407338314
updated units for 407339429
updated building type for 407339429
updated 407339429
updated units for 407339688
updated building type for 407339688
updated 407339688
updated units for 407339706
updated building type for 407339706
updated 407339706
updated units for 407339754
updated building type for 407339754
updated 407339754
updated units for 407339928
updated building type for 407339928
updated 407339928
updated units for 407339931
updated building type for 407339931
updated 407339931
